# Lab Assignment 2: Data Reading and Processing
**Name:** Eva Santana

**Date:** Oct 13 2025

## Introduction
This project focuses on reading, cleaning, and merging three fragmented and partially damaged datasets (`fortune500.csv`, `lines.json`, and `unstructureddata.txt`).  
I standardize their formats, handle missing values, and generate summary statistics such as total data volume, missing entries, and top companies by revenue and profit.

In [1]:
import pandas as pd
import numpy as np
import csv
import json

file_path = ["fortune500.csv", "lines.json", "unstructureddata.txt"]
good_data, bad_data = [], [] 

# -----------------------------------
# Read CSV file safely
# -----------------------------------

with open(file_path[0], mode='r', encoding='utf-8') as file:
    csv_reader = csv.reader(file)
    header = next(csv_reader, None)

    for row in csv_reader:
        if not row or len(row) < 5:
            continue

        year = row[0]
        rank = row[1]
        company = row[2]
        revenue = row[3]
        profit = row[4]

        bad = False

        # Check for missing or NaN values safely
        if pd.isna(year) or pd.isna(rank) or pd.isna(company) or pd.isna(revenue) or pd.isna(profit):
            bad = True
        else: 
            year_val = pd.to_numeric(year, errors = 'coerce')
            rank_val = pd.to_numeric(rank, errors = 'coerce')
            revenue_val = pd.to_numeric(revenue, errors = 'coerce')
            profit_val = pd.to_numeric(profit, errors = 'coerce')

            if pd.isna(year_val) or pd.isna(rank_val) or pd.isna(revenue_val) or pd.isna(profit_val):
                bad = True 

        if bad:
            bad_data.append(row)
        else:
            good_data.append(row)
                
# -----------------------------------
# Read JSON Lines file safely
# -----------------------------------
title = ['Year', 'Rank', 'Company', 'Revenue (in millions)', 'Profit (in millions)']

with open(file_path[1], 'r') as file:
    for line in file: 
        try: 
            json_obj = json.loads(line.strip())

            for key in title: 
                if key not in json_obj:
                    json_obj[key] = None

            row = [
                json_obj['Year'],
                json_obj['Rank'],
                json_obj['Company'],
                json_obj['Revenue (in millions)'],
                json_obj['Profit (in millions)']]
            # Check if any fields are missing or invalid numerically
            bad = False
            if any(pd.isna(val) for val in row):
                bad = True
            else:
                # Try converting numeric fields
                year_val = pd.to_numeric(json_obj['Year'], errors='coerce')
                rank_val = pd.to_numeric(json_obj['Rank'], errors='coerce')
                rev_val = pd.to_numeric(json_obj['Revenue (in millions)'], errors='coerce')
                prof_val = pd.to_numeric(json_obj['Profit (in millions)'], errors='coerce')

                if pd.isna(year_val) or pd.isna(rank_val) or pd.isna(rev_val) or pd.isna(prof_val):
                    bad = True

            if bad:
                bad_data.append(row)
            else:
                good_data.append(row)

        except json.JSONDecodeError:
            # Malformed JSON lines go straight to bad_data
            bad_data.append([line.strip()])         

# -----------------------------------
# Read Unstructured Text file safely
# -----------------------------------
expected_keys = ["Year", "Rank", "Company", "Revenue (in millions)", "Profit (in millions)"]

# Function to process a block of text and return a dictionary
def process_block_with_damage(block):
    entity = {key: None for key in expected_keys}
    for item in block:
        if ": " in item:
            key, value = item.split(": ", 1)  # Split only on the first ': '
            if key in entity:
                entity[key] = value.strip()
    return entity

current_data = []
with open(file_path[2], 'r', encoding='utf-8') as file:
    for line in file:
        if line.strip() == "":
            if current_data:  # Process accumulated block
                processed_data = process_block_with_damage(current_data)

                # Convert to row format
                row = [
                    processed_data["Year"],
                    processed_data["Rank"],
                    processed_data["Company"],
                    processed_data["Revenue (in millions)"],
                    processed_data["Profit (in millions)"]
                ]

                # Validate row (check missing or invalid numerics)
                bad = False
                if any(pd.isna(val) or val is None or val == "" for val in row):
                    bad = True
                else:
                    year_val = pd.to_numeric(processed_data["Year"], errors='coerce')
                    rank_val = pd.to_numeric(processed_data["Rank"], errors='coerce')
                    rev_val = pd.to_numeric(processed_data["Revenue (in millions)"], errors='coerce')
                    prof_val = pd.to_numeric(processed_data["Profit (in millions)"], errors='coerce')

                    if pd.isna(year_val) or pd.isna(rank_val) or pd.isna(rev_val) or pd.isna(prof_val):
                        bad = True

                if bad:
                    bad_data.append(row)
                else:
                    good_data.append(row)

                current_data = []  # Reset for next block
        else:
            current_data.append(line.strip())

    # Handle the last block if file doesn’t end with a blank line
    if current_data:
        processed_data = process_block_with_damage(current_data)
        row = [
            processed_data["Year"],
            processed_data["Rank"],
            processed_data["Company"],
            processed_data["Revenue (in millions)"],
            processed_data["Profit (in millions)"]
        ]
        bad = False
        if any(pd.isna(val) or val is None or val == "" for val in row):
            bad = True
        else:
            year_val = pd.to_numeric(processed_data["Year"], errors='coerce')
            rank_val = pd.to_numeric(processed_data["Rank"], errors='coerce')
            rev_val = pd.to_numeric(processed_data["Revenue (in millions)"], errors='coerce')
            prof_val = pd.to_numeric(processed_data["Profit (in millions)"], errors='coerce')

            if pd.isna(year_val) or pd.isna(rank_val) or pd.isna(rev_val) or pd.isna(prof_val):
                bad = True

        if bad:
            bad_data.append(row)
        else:
            good_data.append(row)

 

df_good = pd.DataFrame(good_data, columns=['Year', 'Rank', 'Company', 'Revenue', 'Profit'])
df_bad = pd.DataFrame(bad_data)




# Result 
- The aggregate data volume (Good data)

This result was found using the len() function on the df_good dataframe, which counts the total number of valid rows.

- Instances of missing data (Bad data)

This result was found using the len() function on the df_bad dataframe, which counts the number of rows that contained missing or invalid entries.

- The number of unique companies

This result was found by accessing the 'Company' column from the df_good dataframe and using the unique() operation to determine how many distinct company names were present in the good dataset.

- The company with the highest revenue from 1995 to 1998

First, I filtered all rows where the 'Year' column was between 1995 and 1998 (inclusive). Then, I located the row with the maximum 'Revenue' value within this range to identify which company had the highest revenue during that period.

- The company with the highest profit from 1995 to 1998

After filtering for the years 1995–1998, I found the row with the maximum 'Profit' value to determine the company that achieved the highest profit within that same time frame.

In [2]:
# -----------------------------------
# Data Statistics Section
# -----------------------------------

# Total number of valid (good) rows
total_good = len(df_good)

# Total number of invalid (bad) rows
total_bad = len(df_bad)

# Number of unique companies
unique_companies = df_good['Company'].nunique()

# Company with highest revenue between 1995–1998
df_good['Year'] = pd.to_numeric(df_good['Year'])
df_good['Revenue'] = pd.to_numeric(df_good['Revenue'])
df_good['Profit'] = pd.to_numeric(df_good['Profit'])

# Filter for 1995-1998
df_95_98 = df_good[(df_good['Year'] >= 1995) & (df_good['Year'] <= 1998)]

# Find top revenue company
top_revenue_row = df_95_98.loc[df_95_98['Revenue'].idxmax()]
top_revenue_company = top_revenue_row['Company']
top_revenue_value = top_revenue_row['Revenue']

# Company with highest profit between 1995–1998
top_profit_row = df_95_98.loc[df_95_98['Profit'].idxmax()]
top_profit_company = top_profit_row['Company']
top_profit_value = top_profit_row['Profit']

# Combine results into a single summary DataFrame
Results_Combine = pd.DataFrame({
    'Metric': [
        'Total Good Data',
        'Total Bad Data',
        'Unique Companies',
        'Top Revenue Company (1995–1998)',
        'Top Revenue Value (in millions)',
        'Top Profit Company (1995–1998)',
        'Top Profit Value (in millions)'
    ],
    'Value': [
        total_good,
        total_bad,
        unique_companies,
        top_revenue_company,
        top_revenue_value,
        top_profit_company,
        top_profit_value
    ]
})

Results_Combine.to_csv("Results_Combine.csv", index=False)


Results_Combine




,Metric,Value
0,Total Good Data,42590
1,Total Bad Data,4759
2,Unique Companies,2359
3,Top Revenue Company (1995–1998),General Motors
4,Top Revenue Value (in millions),178174.0
5,Top Profit Company (1995–1998),Exxon Mobil
6,Top Profit Value (in millions),8460.0
